# Deep-subspace-clustering-networks
My GitHub：https://github.com/Elliot-Wang/Deep-subspace-clustering-networks.git  
My Account：15260223133@163.com

In [1]:
!pip install munkres==1.0.8
#!pip install tensorflow==1.14.0
%tensorflow_version 1.x
!git clone https://github.com/Elliot-Wang/DSC-rebuilt.git
!mv DSC-rebuilt/*.py ./

  Created wheel for munkres: filename=munkres-1.0.8-cp36-none-any.whl size=11236 sha256=900af597200ff40db6bdf8857b3f8e2c7a071186d678566a1806bfb1027f6f00
  Stored in directory: /root/.cache/pip/wheels/9f/61/27/66dfa0b5b370aca085ef8dfbe49b6b6d0763f26c3d82f51e5d
Successfully built munkres
Cloning into 'DSC-rebuilt'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 34 (delta 6), reused 34 (delta 6), pack-reused 0
Unpacking objects: 100% (34/34), done.
